# Setup Inicial para os Notebooks das Usinas

## Histórico de Atualizações

<table>
    <thead>
        <tr>
            <th>Versão</th>
            <th>Data</th>
            <th>Descrição</th>
            <th>Autor</th>
            <th>Email</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <th>1.0</th>
            <td>21/03/2023</td>
            <td>Criação dos ajustes gerais</td>
            <td>Marcel Quintela, Sergio Urzedo Jr</td>
            <td>marcel.quintela@avanade.com, sergiourzedojr@gmail.com</td>
        </tr>
    </tbody>
</table>

# Setup Inicial

In [24]:
# limpar saidas
import os

def clear_output():
    os.system('cls' if os.name == 'nt' else 'clear')

In [ ]:
try:
    from alive_progress import alive_bar
except ModuleNotFoundError:
    !pip install --quiet alive_progress
    from alive_progress import alive_bar

In [25]:
# INSTALAÇÃO DE PACOTE AUXILIAR NA LEITURA DE ARQUIVOS .XLSX
try:
    import xlrd
except ModuleNotFoundError:
    !pip install --quiet xlrd
    import xlrd

clear_output()

In [26]:
# INSTALAÇÃO DE PACOTE DO IBGE PARA COLETAR INFORMAÇÕES GEOGRÁFICAS
#clear_output()
try:
    import ibge 
except ModuleNotFoundError:
    !pip install --quiet ibge
    import ibge     

In [27]:
clear_output()
try:
    import missingno as msno 
except ModuleNotFoundError:
    !pip install --quiet missingno
    import missingno as msno

In [ ]:
# try:
#     import xlwings as xw
# except ModuleNotFoundError:
#     !pip install -q xlwings
#     import xlwings as xw


## Bibliotecas Utilizadas

In [8]:
# Manipulação de dados
import numpy as np
import pandas as pd
import json

# Manipulação e processamento de texto e caracteres
import re
import unicodedata

# Manipulação de data/hora
import time
from time import strftime
from pytz import timezone
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta

# Visualização
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Avisos e logs
import logging
import warnings

# Informações Geográficas
from ibge.localidades import *                              # funções de dados geograficos segundo codificação do IBGE

# Processos de Imputação e Machine Learning
from sklearn.preprocessing import LabelEncoder              # transformar var Categorica em numerica [ordinal ou não]
from sklearn.impute import KNNImputer                       # função de imputação baseada em sua vizinhança

# Prophet para previsão de series de tempo
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly

# Calculo de estatisticas
from scipy import stats

# Medir perdormance dos modelos
from sklearn import metrics


In [ ]:
# Configurar constante de Data Hora
tz_SP = timezone('America/Sao_Paulo')

In [ ]:
# usado para remover as mensagens de output de processamento dos modelos prophet.
cmdstanpy_logger = logging.getLogger("cmdstanpy")
cmdstanpy_logger.disabled = True

In [9]:
# settings pd warnings 

# elimina avisos do tipo:
# A value is trying to be set on a copy of a slice from a DataFrame
# See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

# Ocorrência provável devido a desatualização do pandas

pd.set_option('mode.chained_assignment', None)

# modo de exibição sem Notação Científica
pd.options.display.float_format = '{:.6f}'.format

In [ ]:
class cores:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

In [ ]:
a1 = ('Bibliotecas e configurações gerais:*carregadas com sucesso!\n')

## Definições gerais

### Ambiente

In [10]:
blob_container_name = 'general'                                             # container name

blob_relative_path_raw = 'nuvem/Usinas/MAPA/'                               # relative folder path
blob_relative_path_enriched = 'enriched/mercado_potencial/mapa/'            # relative folder path

linked_service_raw = 'LS_ADLS_RAW_01'                                       # linked service name
linked_service_enriched = 'LS_ADLS_ENRICHED_01'                             # linked service name


In [11]:
ls_raw = mssparkutils.credentials.getPropertiesAll(linked_service_raw)
ls_enriched = mssparkutils.credentials.getPropertiesAll(linked_service_enriched)

converter_dic_raw = json.loads(ls_raw)
converter_dic_enriched = json.loads(ls_enriched)

#coletando o endpoint
end_point_raw = (converter_dic_raw['Endpoint'].split("/"))[2]
end_point_enriched = (converter_dic_enriched['Endpoint'].split("/"))[2] 

#Utilizado na leitura via metodo mssparkutils
abfss_path_raw = 'abfss://%s@%s/%s' % (blob_container_name, end_point_raw, blob_relative_path_raw)
abfss_path_enriched = 'abfss://%s@%s/%s' % (blob_container_name, end_point_enriched, blob_relative_path_enriched)

In [12]:
del(blob_container_name,
    blob_relative_path_raw,
    blob_relative_path_enriched,
    ls_raw,
    ls_enriched,
    converter_dic_raw,
    converter_dic_enriched,
    end_point_raw,
    end_point_enriched)

In [13]:
a2 = ('Configurações de ambiente:*'+ 
      ' - abfss_path_raw: '+ abfss_path_raw +'\n' +
      ' - abfss_path_enriched: '+ abfss_path_enriched + '\n' + 
      'carregadas com sucesso!\n')

# FUNÇÕES

## Funções de imputação de dados faltantes

In [ ]:
# criando validador para o tipo de imputação: TIPO_IMP -  0: UF; 1:REGIAO

def validador_impt (dados, alvo, chave='CNPJ', geo='UF'):
    df = dados.copy()
    df['TIPO_IMP'] = 0                                                               # Todos assumem, a priori, que será por UF
    uf = df[geo].value_counts().index.to_list()
    for i in uf:
        falta = df[df[geo]==i][alvo].isna().sum()/len(df[df[geo]==i])
        if falta >=0.6:                                                             # se 60% dos dados forem faltantes faz por REGIAO
            df.loc[df[geo]==i, 'TIPO_IMP'] = 1

    df.sort_values(chave, ignore_index=True, inplace=True)
    #df.reset_index(drop=True, inplace=True)
    return(df)

In [ ]:
def imputacao(dados, variaveis, chave, geo='UF', viz=5, ind=False):
    df = dados.copy()
    cols = df.columns.drop(variaveis)                                                      # Variáveis auxiliares
    query     = 'TIPO_IMP == 0' if geo=='UF' else 'TIPO_IMP == 1'
    #viz       = 5                                                                         # 2 if geo=='UF' else 5
    loc       = df.query(query)[geo].value_counts().index.to_list()                        # listando as UF
    knn       = KNNImputer(n_neighbors=viz, weights = 'distance', add_indicator=ind)       # modelo e imputação
    result = pd.DataFrame()                                                                # dataframe que acumulará os resultados por UF

    for i in loc:
        aux_1 = df[df[geo]==i][cols].reset_index(drop=True)
        aux_2 = df[df[geo]==i][variaveis].reset_index(drop=True)
        knn.fit(aux_2)
        aux = pd.DataFrame(knn.transform(aux_2), columns=knn.get_feature_names_out())
        aux = aux_1.join(aux)
        result = pd.concat([result, aux], axis=0, ignore_index=True)
    result.sort_values(chave, ignore_index=True, inplace=True)
    return (result)

In [ ]:
def agrega_imputados(dados, variaveis, chave, alvo, nivel, viz, ind):
    dic = {0:'Não', 1:'Sim'}
    df = dados.copy()
    df = validador_impt(df, alvo)
    col = df.columns.drop(variaveis)
    for i in nivel:
        aux = imputacao(df, variaveis, chave, i, viz, ind).set_index(chave)
        df.set_index(chave, inplace=True)
        df.update(aux, overwrite=False)                                                 # Atualiza df
        df.reset_index(drop=False, inplace=True)
    if ind==True:
        df = df.merge(aux.reset_index(drop=False).filter(regex='CNPJ|missing'), how='left', on='CNPJ')          # incluindo colunas indicadoras de imputação caso ind=True    
        df.columns = df.columns.str.replace('missingindicator', lambda x: 'imputado', regex=True)
        df = df.replace({col:dic for col in df.columns if ~col.find('imputado')})
    return(df)

In [ ]:
def imp_anual(df, freq, l, cols, var, ind=False):

    result =pd.DataFrame()
    #l = [x for x in cols if x not in var+['R_SOCIAL','UF','REGIAO']]
    temp = sorted(df[freq].value_counts().index.tolist())
    #print('Percentual de USINAS que a  realizou vendas no', freq)

    for i in temp[:-1]:                                     # até o ultimo periodo[ano, mes] completo
        df_a = df[df[freq]==i][l+['Vol_Total','Freq','Vol_Medio']]
        us = usinas.merge(df_a, how='left', on='CNPJ')
        us[freq]=i
        #print('{}: {:2.2%}'.format(i, us[~us['Freq'].isnull()].shape[0]/len(us)))
        aux = agrega_imputados(us[cols], var, chave='CNPJ', alvo='Vol_Medio', nivel=['UF', 'REGIAO'], viz=5, ind=ind)
        result = pd.concat([result, aux], axis=0, ignore_index=True)
    return (result)

In [ ]:
a3 = ('Funções de imputação de dados faltantes:*'+
        ' - validador_impt (dados, alvo, chave="CNPJ", geo="UF")\n'+
        ' - agrega_imputados(dados, variaveis, chave, alvo, nivel, viz, ind)\n' +
        ' - imp_anual(df, freq, l, cols, var, ind=False)\n'+
        'carregadas com sucesso!\n')

## Funções de previsão com Prophet

In [ ]:
def create_metricas():
    """
        Cria dicionário em branco das métricas dos modelos
    Result:
    --------
        dic_resilt: dict
            dicionário contendo informações basicas das métricas do modelo
    """
    dic_result = {
                    'data':0,
                    'modelo': 0, 
                    'regressores':'' , 
                    'treino_i':0,
                    'treino_f':0,
                    'teste_i':0,
                    'teste_f':0,
                    'metricas':{'MSE':0,'MAE':0,'RMSE':0,'MAPE':0,'R2':0}
                }
    return(dic_result)


In [ ]:
def append_metricas(treino, teste, indice):
    """
        Insere informações das métricas dos modelos no dicionário.
    Parâmetros
    ----------
        treino: pd.DataFrame
            Amostra de treino a ser ajustada
        teste: pd.DataFrame
            Amostra de teste que será comparada com as previsões deo treino
        indice: int
            indice que auxilia na navegação entre os dados do dicionario dos modelos e suas regressoras.
    Result:
    --------
        dic_resilt: dict
            dicionário contendo informações basicas das métricas do modelo
    """
    d_result = create_metricas()
    x = process_modelos(treino, list(dict_var.values())[indice])
    previsto = x[(x['ds']>=teste.index.min()) & (x['ds']<=teste.index.max())]['yhat']
    real = teste[list(dict_var.values())[indice][1]]

    d_result['data']          = datetime.now(tz_SP).strftime("%Y-%m-%d %H:%M:%S")
    d_result['modelo']        = list(dict_var.keys())[indice]
    d_result['regressores']   = list(dict_var.values())[indice]
    d_result['treino_i']      = treino['ds'].min().strftime("%Y-%m-%d") 
    d_result['treino_f']      = treino['ds'].max().strftime("%Y-%m-%d") 
    d_result['teste_i']       = teste['ds'].min().strftime("%Y-%m-%d") 
    d_result['teste_f']       = teste['ds'].max().strftime("%Y-%m-%d")
    d_result['metricas']      = metricas_profecias(real,previsto)


    return(d_result)

In [ ]:
def dividir_train_test(data, num_meses=12):
    '''
    Função que divide os dados a serem usados pelos modelos em treino e teste conforme o
    padrão pd.DataFrame Prothet contendo as colunas ['ds','y']

    Parâmetros
    ----------
        data: pd.Dataframe
            dataframe em formato time series do Prothet a ser dividido
        num_meses: int
            número de meses usados para teste (default 12 meses)
    Retorno
    -------:
        train, test: pd.Dataframe
            Dataframes de treino e teste
    '''
    train_data_fim = data['ds'].max() - relativedelta(months=num_meses)
    # divisão dos datasets de Treino: train e Teste:test
    train = data[data['ds'] <= train_data_fim]
    test = data[data['ds'] > train_data_fim]

    return (train, test)

In [ ]:
def metricas_profecias(y_real, y_prev, saida='dict', nm_modelo=''):
    '''
    Calcula as seguintes métricas :
        - MSE, MAE, RMSE e MAPE

    Parâmetros
    ----------
        y_real: float64
            Valores reais de Y amostra ou toda a série 
        y_prev (float64)
            Valores previstos de Y, informar pd.DataFrame ou Series contendo ['yhat'] que a função resgata
            a parte equivalente ao teste
        saida: str
            Formato da saída [print ou dicionario]
        nm_modelo:str
            Nome do modelo

    Retorna:
        MSE, MAE, RMSE e MAPE: dict ou print
            Conjunto das métricas do modelo testado
    '''
    try:
        if isinstance(y_prev, pd.DataFrame):
            y_prev = y_prev['yhat'][-len(y_real):]
        elif isinstance(y_prev, pd.Series) and y_prev.name == 'yhat':
            y_prev = y_prev[-len(y_real):]
    except:
        return (print('Parâmetros fora do padrão!\n'), help(metricas_profecias))
    
    metricas = {'MSE':metrics.mean_squared_error(y_real, y_prev),
                'MAE':metrics.mean_absolute_error(y_real, y_prev),
                'RMSE':np.sqrt(metrics.mean_squared_error(y_real, y_prev)),
                'MAPE':metrics.mean_absolute_percentage_error(y_real, y_prev)}
                #'R2':metrics.r2_score(y_teste, y_prev)}

    if saida == 'dict':
        return(metricas)
    elif saida == 'print':
        print(cores.BOLD + '-'*40)
        print('          Métricas de Avaliação')
        print(nm_modelo)
        print('-'*40 + cores.END)
        print(' • MSE : {:.5f}'.format(metricas['MSE']))
        print(' • MAE : {:.5f}'.format(metricas['MAE']))
        print(' • RMSE: {:.5f}'.format(metricas['RMSE']))
        print(' • MAPE: {:.5f}'.format(metricas['MAPE']))
       # print(' • R²  : {:.5f}\n'.format(metricas['R2']))
    else:
        return(print('Parâmetros fora do padrão!\n'), help(metricas_profecias))

In [ ]:
def modelo_final(dados, periodos=12):
    r"""Ajusta modelo prophet de serie temporal.

        Define, ajusta e prevê modelo de serie temporal baseado na biblioteca prophet.
        Modelo com a possibilidade de inclusão de variáveis regressoras. 
        
    Parameters
    ----------
    dados: pd.DataFrame 
        Dataframe com variáveis exigidas para a criação do modelo prophet.
        No dataframe deve conter, obrigatoriamente, coluna 'ds': datatime da série e 
        'y': variável alvo do modelo.
        As variáveis regressoras, quando existire, devem estar presentes no mesmo dataframe.
    periodos: int
        Períodos, em meses, a serem previstos pelo modelo.
        Por padrão, doi feinido 12 meses a serem previstos.
    
    Returns
    -------
    previsao: pd.DataFrame
        Dataframe com os resultados do predict model adcionada das variáveis de input do modelo.
    modelo: prophet model
        Modelo prophet ajustado (fitted) 
    """

    regressores = dados.columns.to_list()[2:]       # os 2 primeiros são DataTime e o Y da serie temporal

    lst_results = ['ds','yhat','yhat_lower', 'yhat_upper',
                   'trend', 'trend_lower', 'trend_upper',
                   'yearly', 'yearly_lower', 'yearly_upper']
    
    modelo = Prophet(seasonality_mode='multiplicative',
                     yearly_seasonality=True, 
                     weekly_seasonality=False,
                     daily_seasonality=False,
                     changepoint_range=0.8,
                     changepoint_prior_scale=0.05)

    for i in range(len(regressores)):               # passeia por cada lista agregativa
        modelo.add_regressor(regressores[i], standardize=False)
                
    modelo.fit(dados)

    futuro = modelo.make_future_dataframe(periods=periodos, freq='MS') #MS início do mês 
    futuro = pd.merge(futuro, dados, on='ds', how='outer')
    futuro = futuro.fillna(method='ffill')
    
    previsao = modelo.predict(futuro)
    previsao = pd.merge(previsao[lst_results], dados, on='ds',how='outer') 

    return(previsao, modelo)

In [ ]:
def process_modelos(dados, lista):
    """Processa o modelo e prepara os resultados para salvaguarda.

        Auxilia no processamento em lote de todos os modelos definidos. 
        
    Parâmetros
    ----------
    dados: pd.DataFrame
        dataframe com as informações a serem previstas
    lista: lista de strings 
        Lista com o nome das váriaveis escolhidas para o modelo    
    
    Retorno
    -------
    previsao: pd.DataFrame
        Dataframe com os resultados do predict model adcionada das variáveis de input do modelo.
        Nele a variável alvo do modelo "y" retoma o nome original do dataset, com o prefixo 'y_'
        deixando clara que as demais informações 'yhat', 'trend', 'yearly' e seus intervalos de confiança
        foram obtidos por meio do 'y_' e demais regressoras do dataset. 
    """
    data = dados[lista]
    data.reset_index(drop=True,inplace=True)
    #data.columns = data.columns.str.replace('data', 'ds')
    data.columns = data.columns.str.replace(lista[1], 'y')
    
    previsao, modelo  = modelo_final(data)
    previsao.insert(1, 'y_'+lista[1], previsao['y'])
    previsao.drop('y',axis=1,inplace=True)

    return(previsao)

In [7]:
a4 = ('Funções de para os modelos Prophet:*'+
        ' - create_metricas()\n'+
        ' - append_metricas(treino, teste, indice)\n'+
        ' - dividir_train_test(data, num_meses=12)\n'+
        ' - metricas_profecias(y_real, y_prev, saida="dict", nm_modelo="")\n'+
        ' - modelo_final(dados, periodos=12)\n'+
        ' - process_modelos(dados, lista)\n'+
        'carregadas com sucesso!\n')

In [ ]:
# multi_plot requires two variables:
# - df is a dataframe with stocks as columns and rows as date of the stock price
# - addAll is to have a dropdown button to display all stocks at once

def multi_plot(df, titulo='Título', legenda='Legenda', addAll = True):
    fig = go.Figure()

    for column in df.columns.to_list():
        fig.add_trace(
            go.Scatter(
                x = df.index,
                y = df[column],
                name = column
            )
        )

    button_all = dict(label = 'All',
                      method = 'update',
                      args = [{'visible': df.columns.isin(df.columns),
                               'title': 'All',
                               'showlegend':True}])

    def create_layout_button(column):
        return dict(label = column,
                    method = 'update',
                    args = [{'visible': df.columns.isin([column]),
                             'title': column,
                             'showlegend': True}])

    fig.add_annotation(
            showarrow=False,
            text='Gerado em ' + dt.now(tz_SP).strftime("%d %b %Y às %H:%M:%S"),
            font=dict(size=12),
            align="right",
            x=1.0,
            y=-0.35,
            xref='paper',
            yref='paper',
            xanchor='right',
            yanchor='bottom',)
    
    fig.update_xaxes(matches='x', rangeslider_visible=True, rangeslider_thickness = 0.05)
    fig.update_xaxes(rangeslider= {'visible':True})

    fig.update_layout(
        template='plotly_white', # template ["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none"]
        title = titulo,
        legend_title = legenda,
        updatemenus=[go.layout.Updatemenu(
            active = 0,
            buttons = ([button_all] * addAll) + list(df.columns.map(lambda column: create_layout_button(column)))
            )
        ])
    
    fig.show()

# Gráficos

In [ ]:
a5 = ('Funções de apresentação gráfica:*'+
        ' - multi_plot(df, titulo="Título", legenda="Legenda", addAll = True)\n'+
#        ' - process_modelos (dados, lista)\n'+
        'carregadas com sucesso!\n')

In [ ]:
for x in [a1,a2,a3,a4,a5]:
    print(x.split('*')[0])
    with alive_bar(100+len(x)*5, force_tty=True) as bar:  # declare your expected total    
        for item in range(100+len(x)*5):        # <<-- your original loop
            time.sleep(.001)
            bar()                 # call `bar()` at the end
    print(x.split('*')[1])                        # your actual processing here
del(a1,a2,a3,a4,a5)